In [1]:
import random
import time
import math
from progress0316.bar import ProgressBar

<h2>Util functions

In [2]:
#prende unna posizione e fa un cambio random su quella posizione
def tweak(positions):
    l = len(positions)
    x = random.randint(0,l-1)
    y = random.randint(0,l-1)
    while(x==y):
        y = random.randint(0,l-1)
    positions[x],positions[y] = positions[y],positions[x]

#prende una posizione e riporta la posizione con un tweak random
def makeTweak(positions):
    tweaked = positions.copy()
    tweak(tweaked)
    return tweaked

#fa un po di shuffle random della posizione
def randomStart(dim):
    positions = list(range(dim))
    for _ in range(100):
        tweak(positions)
    return positions

#verifica se la posizione è sulla scacchiera
def isInBoard(position, dim):
    x,y = position
    return (0<=x<dim and 0<=y<dim)

#conta il numero di hits delle regine sulla scacchiera
#(possono stare solo sulle diagonali)
def hits(positions):
    _hits = 0
    dim = len(positions)
    for i in range(dim):
        #si sposta sulla diagonale in alto a destra
        (x,y) = (i+1, positions[i]+1)
        while isInBoard((x,y), dim):
            if positions[x] == y:
                _hits+=1
            x+=1
            y+=1
        #si sposta sulla diagonale in basso a destra
        (x,y) = (i+1, positions[i]-1)
        while isInBoard((x,y), dim):
            if positions[x] == y:
                _hits+=1
            x+=1
            y-=1
        #si sposta sulla diagonale in alto a sinistra
        (x,y) = (i-1, positions[i]+1)
        while isInBoard((x,y), dim):
            if positions[x] == y:
                _hits+=1
            x-=1
            y+=1
        (x,y) = (i-1, positions[i]-1)
        #si sposta sulla diagonale in basso a sinistra
        while isInBoard((x,y), len(positions)):
            if positions[x] == y:
                _hits+=1
            x-=1
            y-=1
    return _hits

#stampa la scacchiera con le regine
def printBoard(positions):
    for i in range(len(positions)):
        temp = ""
        for j in range(len(positions)):
            if positions[j]==i:
                temp+=" 0 "
            else:
                temp+=" + "
        print(temp)

<h2>Standard hill climbing (deepest ascent)

In [3]:
def lowestNextHits(currentBest, currentHits):
    l = len(currentBest)
    for i in range(0, l-1):
        for j in range(i+i, l):
            temp = currentBest.copy()
            temp[i], temp[j] = temp[j], temp[i]
            _hits = hits(temp)
            if _hits<currentHits:
                currentBest = temp
                currentHits = _hits
    return currentBest, currentHits

def hillClimbing(dim):
    currentState = randomStart(dim)
    currentHits = hits(currentState)
    keepExploring = True
    while keepExploring:
        nextState, nextHits = lowestNextHits(currentState, currentHits)
        if nextHits >= currentHits:
            keepExploring = False
        currentState = nextState
        currentHits = nextHits
    return currentState

positions = hillClimbing(8)
print("solution:", positions)
print("hits:", hits(positions))

solution: [2, 7, 6, 3, 0, 4, 1, 5]
hits: 2


<h2>Random restart Hill climbing

In [4]:
def randomRestart(dim):
    while True:
        currentState = hillClimbing(dim)
        currentHits = hits(currentState)
        if currentHits==0:
            return currentState

positions = randomRestart(8)
print("solution:", positions)
print("hits:", hits(positions))

solution: [5, 0, 4, 1, 7, 2, 6, 3]
hits: 0


<h2>Stochastic hill climbing

In [5]:
def stochasticHill(dim, t):
    currentState = randomStart(dim)
    currentHits = hits(currentState)

    bestState = currentState.copy()
    bestHits = currentHits
    while bestHits>0:
        nextState = makeTweak(currentState)
        nextHits = hits(nextState)

        if nextHits < bestHits:
            bestState = nextState
            bestHits = nextHits
        dE = nextHits - currentHits
        p = 1/(1+math.exp(dE/t))

        if (random.random() < p):
            currentState = nextState
            currentHits = nextHits
    return bestState

positions = stochasticHill(8, 20)
print("solution:", positions)
print("hits:", hits(positions))

solution: [2, 5, 7, 0, 3, 6, 4, 1]
hits: 0


<h2>Simulated annealing

In [6]:
def simulatedAnnealing(dim, iterations, startingT, endingT, alfa):
    currentState = randomStart(dim)
    currentHits = hits(currentState)

    bestState = currentState.copy()
    bestHits = currentHits
    t = startingT
    while(t>endingT and bestHits>0):
        i = 0
        while(i<iterations and bestHits>0):
            i+=1
            nextState = makeTweak(currentState)
            nextHits = hits(nextState)
            if (nextHits<currentHits):
                currentState = nextState
                currentHits = nextHits
                if (currentHits < bestHits):
                    bestState = currentState
                    bestHits = currentHits
            else:
                dE = nextHits - currentHits
                p = math.exp(-dE/t)
                if (random.random() <= p):
                    currentState = nextState
                    currentHits = nextHits
        t *= alfa
    return bestState

positions = simulatedAnnealing(8, 100, 30, 0.2, 0.99)
print("solution:", positions)
print("hits:", hits(positions))

solution: [5, 2, 6, 1, 7, 4, 0, 3]
hits: 0


<h2>Tabu search base

In [7]:
import numpy as np

def tabuTenureStart(dim):
    tabuTenure = dict()
    for i in range(dim-1):
        for j in range(i+1, dim):
            tabuTenure[(i,j)] = 0    
    return tabuTenure

#true if la mossa è legittima
def tabuCheck(tabuTenure, move):
    return tabuTenure[move]==0

def tabuDecrement(tabuTenure):
    for key in tabuTenure.keys():
        if tabuTenure[key] != 0:
            tabuTenure[key]-=1

def applyMove(state, move):
    state = state.copy()
    state[move[0]], state[move[1]] = state[move[1]], state[move[0]]
    return state

def tabuSearch(dimension, tabuBan = 5):
    currentState = np.array(randomStart(dimension))
    currentHits = hits(currentState)

    bestState = currentState.copy()
    bestHits = currentHits

    tabuTenure = tabuTenureStart(dimension)
    while (bestHits > 0):
        allMoves = list(filter(lambda move: tabuCheck(tabuTenure,move), list(tabuTenure.keys()))) #tolgo tutte le mosse tabu
        bestMove = allMoves.pop()
        bestNext = applyMove(currentState, bestMove)
        bestNextHits = hits(bestNext)
        for move in allMoves:
            temp = applyMove(currentState, move)
            tempHits = hits(temp)
            if tempHits<bestNextHits:
                bestMove = move
                bestNext = temp
                bestNextHits = tempHits
        currentState = bestNext
        currentHits = bestNextHits
        tabuDecrement(tabuTenure)
        tabuTenure[bestMove] = tabuBan
        if (currentHits<bestHits):
            bestState = currentState
            bestHits = currentHits
    return bestState

positions = tabuSearch(8)
print("solution:", positions)
print("hits:", hits(positions))

solution: [3 5 0 4 1 7 2 6]
hits: 0


<h2>Tabu search con aspiration criterion

In [8]:
def tabuSearchAC(dimension, tabuBan = 5):
    currentState = np.array(randomStart(dimension))
    currentHits = hits(currentState)

    bestState = currentState.copy()
    bestHits = currentHits

    tabuTenure = tabuTenureStart(dimension)
    allMoves = list(tabuTenure.keys())
    while (bestHits > 0):
        bestMove = None
        bestNext = None
        bestNextHits = None
        for move in allMoves:
            tempState = applyMove(currentState, move)
            tempHits = hits(tempState)
            if tabuCheck(tabuTenure, move):
                if bestNext is None:
                    bestMove = move
                    bestNext = applyMove(currentState, move)
                    bestNextHits = hits(bestNext)
                else:
                    if tempHits<bestNextHits:
                        bestMove = move
                        bestNext = tempState
                        bestNextHits = tempHits
            else:
                if tempHits < bestHits:
                    bestMove = move
                    bestNext = tempState
                    bestNextHits = tempHits
                
        currentState = bestNext
        currentHits = bestNextHits
        tabuDecrement(tabuTenure)
        tabuTenure[bestMove] = tabuBan
        if (currentHits<bestHits):
            bestState = currentState
            bestHits = currentHits
    return bestState

positions = tabuSearchAC(8)
print("solution:", positions)
print("hits:", hits(positions))

solution: [4 0 7 3 1 6 2 5]
hits: 0


<h2>Tabu search con frequency based memory

In [9]:
def updateFBM(fbm, lastMoves, move):
    if len(lastMoves) == 50:
        lastMove = lastMoves.pop(0)
        fbm[lastMove] -= 1
    lastMoves.append(move)
    fbm[move] += 1

def penalty(fbm, move):
    return 0.5*(fbm[move])

def startFBM(dim):
    return tabuTenureStart(dim)

def tabuSearchFBM(dimension, tabuBan = 5):
    currentState = np.array(randomStart(dimension))
    currentHits = hits(currentState)

    bestState = currentState.copy()
    bestHits = currentHits

    tabuTenure = tabuTenureStart(dimension)
    fbm = startFBM(dimension)
    lastMoves = []
    while (bestHits > 0):
        allMoves = list(filter(lambda move: tabuCheck(tabuTenure,move), list(tabuTenure.keys()))) #tolgo tutte le mosse tabu
        bestMove = allMoves.pop()
        bestNext = applyMove(currentState, bestMove)
        bestNextHits = hits(bestNext) - penalty(fbm, bestMove)
        for move in allMoves:
            temp = applyMove(currentState, move)
            tempHits = hits(temp) - penalty(fbm, move)
            if tempHits<bestNextHits:
                bestMove = move
                bestNext = temp
                bestNextHits = tempHits
        
        currentState = bestNext
        currentHits = bestNextHits
        tabuDecrement(tabuTenure)
        tabuTenure[bestMove] = tabuBan
        updateFBM(fbm, lastMoves, bestMove)
        if (currentHits<bestHits):
            bestState = currentState
            bestHits = currentHits
    return bestState

positions = tabuSearchFBM(8)
print("solution:", positions)
print("hits:", hits(positions))

solution: [3 1 6 2 5 7 0 4]
hits: 0


<h2>Efficienze algoritmi a paragone

In [10]:
iterationsPerAlgorithm = 100
dim = 10

#random restart hill climb
i=0
print("random restart hill climb")
start = time.time()
bar = ProgressBar(iterationsPerAlgorithm)
while i<iterationsPerAlgorithm:
    bar.next()
    result = randomRestart(dim)
    i+=1
end = time.time()
print("time per climb:",round((end-start)/iterationsPerAlgorithm, 5), "seconds")
print()

#stochastic hill climb
#fa mezzo schifo

#simulated annealing


#tabu search
i=0
print("tabu search")
bar.reset(iterationsPerAlgorithm)
start = time.time()
while(i<iterationsPerAlgorithm):
    bar.next()
    result = tabuSearch(dim)
    i+=1
end = time.time()
print("time per climb:",round((end-start)/iterationsPerAlgorithm, 5), "seconds")
print()

#tabu search aspiration criterion
i=0
print("tabu search aspiration criterion")
bar.reset(iterationsPerAlgorithm)
start = time.time()
while(i<iterationsPerAlgorithm):
    bar.next()
    result = tabuSearchAC(dim)
    i+=1
end = time.time()
print("time per climb:",round((end-start)/iterationsPerAlgorithm, 5), "seconds")
print()

#tabu search frequency based memory
i=0
print("tabu search frequency based memory")
bar.reset(iterationsPerAlgorithm)
start = time.time()
while(i<iterationsPerAlgorithm):
    bar.next()
    result = tabuSearchFBM(dim)
    i+=1
end = time.time()
print("time per climb:",round((end-start)/iterationsPerAlgorithm, 5), "seconds")

random restart hill climb
|████████████████████████████████████████| [100.00]% (total time: 2 seconds)                    
time per climb: 0.02842 seconds

tabu search
|████████████████████████████████████████| [100.00]% (total time: 2 seconds)                    
time per climb: 0.02762 seconds

tabu search aspiration criterion
|████████████████████████████████████████| [100.00]% (total time: 2 seconds)                    
time per climb: 0.02662 seconds

tabu search frequency based memory
|████████████████████████████████████████| [100.00]% (total time: 2 seconds)                    
time per climb: 0.0218 seconds


In [11]:
solutions = []

In [12]:
iTot = 100
i=0
def isEqual(s1,s2):
    if len(s1) != len(s2):
        return False
    for a,b in zip(s1,s2):
        if a!=b: return False
    return True

while(i<iTot):
    result = list(tabuSearch(8))
    present = False
    for s in solutions:
        if isEqual(s,result):
            present = True
            break
    if not present:
        solutions.append(result)
    i+=1
print(len(solutions))

65
